In [376]:
from openai import OpenAI
import os
import json
from pydantic import BaseModel,Field
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from tqdm import tqdm
import re
import textwrap
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from openai import OpenAI
from pydantic import BaseModel
import traceback
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed

In [377]:
file_path = '../data/GSM8K/test.jsonl'

with open(file_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

In [378]:
load_dotenv()
model = init_chat_model("gpt-4.1-mini", model_provider="openai", temperature=0.2)

In [379]:
class State(TypedDict):
    question: str
    program: Optional[str]
    result: Optional[str]
    final_answer: Optional[str]
    error: Optional[str]
class IntermediateProgram(BaseModel):
    program: str



In [380]:
def pot_node(state: State) -> State:
    pot_messages = [
        SystemMessage('''You will write python program to solve math problems. You will only write code blocks.'''),
        HumanMessage(content=f'''
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
```
def solver():
    """Olivia has $23. She bought five bagels for $3 each. How much money does she have left?"""
    money_initial = 23
    bagels = 5
    bagel_cost = 3
    money_spent = bagels * bagel_cost
    money_left = money_initial - money_spent
    ans = money_left
    return ans
result = solver()
```

Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
```
def solver():
    """Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?"""
    golf_balls_initial = 58
    golf_balls_lost_tuesday = 23
    golf_balls_lost_wednesday = 2
    golf_balls_left = golf_balls_initial - golf_balls_lost_tuesday - golf_balls_lost_wednesday
    ans = golf_balls_left
    return ans
result = solver()            
```
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
```
def solver():
    """There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?"""
    computers_initial = 9
    computers_per_day = 5
    num_days = 4  # 4 days between monday and thursday
    computers_added = computers_per_day * num_days
    computers_total = computers_initial + computers_added
    ans = computers_total
    return ans
result = solver()
```
How about this question?
Q: {state["question"]}
# If the final result is a decimal ending in .0, convert it to an integer before returning.
# Call the solver function and 'MUST' assign the variale 'result'.
# Before returning the final result, DOUBLE-CHECK each variable assignment and calculation to ensure they match the problem statement.
''')]

    model_pot = model.with_structured_output(IntermediateProgram)
    model_invoke=model_pot.invoke(pot_messages)
    code=model_invoke.program
    return {**state, "program": code}

def exec_node(state: State) -> State:
    try:
        exec_globals = {}
        exec(state["program"], {}, exec_globals)
        result = exec_globals.get("result", None)
        if result is None:
            raise ValueError("Missing `result`")
        return {**state, "result": str(result), "error": None}
    except Exception as e:
        return {**state, "result": None, "error": str(e)}

def check_eos(state: State) -> bool:
    if state["error"] is None:
        return True
    else:
        return False
def write_final_answer_node(state:State)->State:

    if state["error"] is None:
        result=str(state["result"])
    else:
        result=str(9999)
    return {**state,"final_answer":result}


builder = StateGraph(State)
builder.add_node("PoT", pot_node)
builder.add_node("Exec", exec_node)

builder.add_node("write_final_answer",write_final_answer_node)

builder.set_entry_point("PoT")
builder.add_edge("PoT", "Exec")
builder.add_edge("Exec", "write_final_answer")
builder.add_edge("write_final_answer",END)
graph = builder.compile()


In [381]:
def extract_ground_truth(answer: str) -> str:
    match = re.search(r"####\s*([\d,]+)", answer)
    if match:
        # Loại bỏ dấu phẩy để đảm bảo kết quả là số chuẩn
        return match.group(1).replace(",", "").strip()
    return ""

def compare_answers(predicted: str, actual: str, eps: float = 1e-3) -> bool:
    try:
        pred = round(float(predicted.strip()))
        act = round(float(actual.strip()))
        return abs(pred - act) < eps
    except ValueError:
        return predicted.strip().lower() == actual.strip().lower()

In [382]:
def process_item(item):
    question = item["question"]
    true_answer = extract_ground_truth(item["answer"])
    try:
        result = graph.invoke({"question": question})
        return {
            "question": question,
            "program":result["program"],
            "true_answer": true_answer,
            "predicted_answer": result["final_answer"],
            "correct": compare_answers(result["final_answer"], true_answer)
        }
    except Exception as e:
        return {"error": str(e), "question": question}

results = []
correct = 0
total = len(dataset[:300])
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(process_item, item) for item in dataset[:300]]
    for future in tqdm(as_completed(futures), total=total):
        result = future.result()
        if "error" not in result:
            results.append(result)
            if result["correct"]:
                correct += 1
        else:
            print(f"Error on question: {result['question'][:60]}... => {result['error']}")
accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}% ({correct}/{total})")


100%|██████████| 300/300 [00:00<00:00, 36100.74it/s]

Error on question: Isabelle bought party supplies for her little sister’s birth... => Tracing using LangChainTracerV1 is no longer supported. Please set the LANGCHAIN_TRACING_V2 environment variable to enable tracing instead.
Error on question: John buys 2 pairs of shoes for each of his 3 children.  They... => Tracing using LangChainTracerV1 is no longer supported. Please set the LANGCHAIN_TRACING_V2 environment variable to enable tracing instead.
Error on question: 4 adults and 8 children are to share 8 packets of chocolate ... => Tracing using LangChainTracerV1 is no longer supported. Please set the LANGCHAIN_TRACING_V2 environment variable to enable tracing instead.
Error on question: Dax went to their farm to pick some apples and found half as... => Tracing using LangChainTracerV1 is no longer supported. Please set the LANGCHAIN_TRACING_V2 environment variable to enable tracing instead.
Error on question: On Mondays, Wednesdays, and Fridays, college student Kimo ha... => Tracing us

In [383]:
output_path = "PaL_results.json"
os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
def custom_encoder(obj):
    if hasattr(obj, "model_dump"):
        return obj.model_dump()
    elif hasattr(obj, "dict"):
        return obj.dict()
    else:
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

wrong_answers = [r for r in results if not r.get("correct", False)]

# Ghi file JSON
try:
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(wrong_answers, f, ensure_ascii=False, indent=2, default=custom_encoder)
    print(f"Đã lưu {len(wrong_answers)} kết quả sai vào {output_path}")
except TypeError as e:
    print(f"Lỗi khi ghi file JSON: {e}")


Đã lưu 0 kết quả sai vào PaL_results.json
